In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import Window, functions as F, types as T

In [2]:
os.environ['PYSPARK_PYTHON']= sys.executable
spark = (
    SparkSession.builder
    .master("local[1]")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "1")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

24/09/23 12:13:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
schema = T.StructType([
    T.StructField('camp_id', T.StringType()),
    T.StructField('control_group_flg', T.ShortType()),
    T.StructField('revenue', T.DoubleType()),
    T.StructField('margin', T.DoubleType()),
    T.StructField('conversion', T.ShortType()),
    T.StructField('user_id', T.StringType()),
])

statistics_df = spark.createDataFrame(pd.read_csv('data_sample.csv'))
statistics_df.show(5)

[Stage 0:>                                                          (0 + 1) / 1]

+-------+-----------------+-------+------+----------+--------------------+
|camp_id|control_group_flg|revenue|margin|conversion|             user_id|
+-------+-----------------+-------+------+----------+--------------------+
|ED_3755|                0|   4.77|   5.3|         1|6f9be979ec9f4aba9...|
|ED_3755|                0|   7.53|  8.12|         1|6a0a4bdd10e670717...|
|ED_3755|                0|  12.33|  5.24|         1|2b96123b22c810b53...|
|ED_3755|                0|   5.44|  4.32|         1|06d37e5b5d6c45a9a...|
|ED_3755|                0|    0.0|   0.0|         0|86fcb416dd0d2b9e7...|
+-------+-----------------+-------+------+----------+--------------------+
only showing top 5 rows



In [4]:
# Список ключевых столбцов
KEY_COLS = ['camp_id']
# Список метрик, по которым считаем статистику
STATISTICS_COLS = ['revenue', 'margin', 'conversion']
ALPHA = 0.05
BS_ITERS = 10000

# Определяем схему дф, который будет возвращаться из pandas_udf,
schema = T.StructType(
    # Ключевые поля
    [T.StructField(i, T.StringType()) for i in KEY_COLS]
    +
    # Схема для левой и правой границ доверительного интервала всех метрик
    [T.StructField(metric, T.ArrayType(T.DoubleType()))
     for metric in STATISTICS_COLS]
)

In [5]:
# Создаем udf для расчета статистик по нашей схеме
@F.pandas_udf(schema, functionType=F.PandasUDFType.GROUPED_MAP)
def bs_on_executor(pdf: pd.DataFrame) -> pd.DataFrame:
    """Функция расчета доверительного интервала для нескольких метрик"""

    # На каждом воркере выставляем переменную ARROW_PRE_0_15_IPC_FORMAT
    # Это нужно для того, чтобы на нашей версии PySpark нормально работал более новый PyArrow
    os.environ['ARROW_PRE_0_15_IPC_FORMAT'] = '1'
    # получаем значение KEY_COLS для того, чтобы их вернуть в ответе
    keys = pdf[KEY_COLS].iloc[0].tolist()

    # контрольная группа
    a = (pdf.loc[pdf['control_group_flg'] == 1]
         .reset_index(drop=True)[STATISTICS_COLS])
    # пилотная группа
    b = (pdf.loc[pdf['control_group_flg'] == 0]
         .reset_index(drop=True)[STATISTICS_COLS])
    len_a = len(a)
    len_b = len(b)
    
    diff_list = []
    # непосредственно само бутстрапирование
    for _ in range(BS_ITERS):
        a_boot = a.sample(len_a, replace=True).mean()
        b_boot = b.sample(len_b, replace=True).mean()
        diff_list.append(b_boot - a_boot)
    bs_result = pd.concat(diff_list)
    
    # расчет доверительного интервала
    ci_res = bs_result.groupby(bs_result.index).quantile([ALPHA/2, 1-ALPHA/2])

    return pd.DataFrame(
        [keys + [ci_res[metric].tolist() for metric in STATISTICS_COLS]])

In [6]:
# Сама группировка и вызов бутстрапа
result = statistics_df.groupBy(*KEY_COLS).apply(bs_on_executor)

In [7]:
# В итоге получаем df с доверительными интервалами метрик
result.toPandas()

,camp_id,revenue,margin,conversion
0,ED_3755,"[0.16969908831748154, 1.2289019013882871]","[-0.8191781309097076, 0.29519699257670146]","[0.011790179957500424, 0.06894876271658001]"


In [8]:
spark.stop()